In [1]:
import pandas as pd
import os, pickle
import string
import re
import numpy as np
from sklearn import linear_model
import sklearn.metrics as mcs
from collections import defaultdict
import time
from tqdm import tqdm as timer

### Director rating dictionary

In [2]:
cast_movie_dict = pickle.load(open("cast_movie_dict.pkl", "rb" ))
movie_rating_dict = pickle.load(open("movie_rating_dict.pkl", "rb" ))

In [3]:
outfile = open("actor_movie_edgelist.txt", "w")

cast_avg_rating_dict = {}
for i, movies in timer(cast_movie_dict.items(), total=len(cast_movie_dict)):
    sum_rating = 0
    count = 0
    for j in movies:
        if j in movie_rating_dict:
            sum_rating += float(movie_rating_dict[j])
            count += 1
        else:
            pass
    if count > 0:
        weight = sum_rating/count
        cast_avg_rating_dict[i] = weight
        for k in movies:
            line = i + '\t' + k + '\t' + str(weight) + '\n'
            outfile.write(line)

outfile.close()
pickle.dump(cast_avg_rating_dict, open("cast_avg_rating_dict.pkl", "wb"))

100%|██████████| 113129/113129 [00:16<00:00, 6846.95it/s]


In [8]:
A2R = pd.DataFrame(list(cast_rating_dict.items()), columns=['actor', 'rating'])
print (A2R)

                           actor    rating
0                      $hort,Too  6.511111
1                      2LiveCrew  5.900000
2                         50Cent  6.003226
3                      A.,Sergey  7.588462
4                      A.S.,Alwi  2.000000
5                  A.V.,SubbaRao  6.822222
6                   Aabel,Per(I)  6.287500
7                      Aaker,Lee  6.616667
8                 Aaltonen,Kosti  5.547059
9                  Aaltonen,Remu  5.611111
10                Aaltonen,Risto  5.455556
11            Aamodt,ThorMichael  5.211111
12                Aanensen,Peter  6.353846
13                  AaronII,Tony  5.740000
14                    Aaron,Aash  5.477778
15                Aaron,Mark(II)  7.471429
16                Aaron,Nicholas  5.981818
17                 Aaron,Quinton  5.691667
18                         Aarya  6.364286
19                      Aas,Arne  5.390000
20                    Aasen,John  6.572727
21                    Aasie,C.S.  6.155556
22         

In [6]:
cast_avg_rating_dict = pickle.load(open("cast_avg_rating_dict.pkl", "rb" ))
movie_cast_dict = pickle.load(open("movie_cast_dict.pkl", "rb" ))
movie_rating_dict = pickle.load(open("movie_rating_dict.pkl", "rb" ))
movie_director_dict = pickle.load(open("movie_director_dict.pkl", "rb" ))
training_movies = pickle.load(open("training_movies.pkl", "rb" ))

In [7]:
training_rating = []
for movie in timer(training_movies):
    training_rating.append(float(movie_rating_dict[movie]))
rating_vector = np.array(training_rating)

100%|██████████| 83533/83533 [00:00<00:00, 368442.65it/s]


In [8]:
prediction = []
def average_actor_rating(mv):
    actors = movie_cast_dict[mv]
    actor_ratings = 0
    count = 0
    for actor in actors:
        if actor in cast_avg_rating_dict:
            actor_ratings += float(cast_avg_rating_dict[actor])
            count += 1
    return (actor_ratings/count)

for mv in timer(training_movies):
    prediction.append(average_actor_rating(mv))

100%|██████████| 83533/83533 [00:02<00:00, 38269.41it/s]


In [9]:
mcs.mean_squared_error(rating_vector, prediction)

0.974611441307004

In [10]:
test_movies = [
    "Batman v Superman: Dawn of Justice (2016)",
    "Mission: Impossible - Rogue Nation (2015)",
    "Minions (2015)"
]

In [11]:
prediction_test_movie = []
for mv in test_movies:
    prediction_test_movie.append(float(average_actor_rating(mv)))
    print ("%s: %.2f" %(mv,average_actor_rating(mv)))

Batman v Superman: Dawn of Justice (2016): 6.50
Mission: Impossible - Rogue Nation (2015): 6.49
Minions (2015): 6.87


In [12]:
real_rating_test = [6.6, 7.4, 6.4]
mcs.mean_squared_error(real_rating_test, prediction_test_movie)

0.35329849383673384

In [41]:
prediction = []
def average_actor_rating(mv):
    actors = movie_cast_dict[mv]
    actor_ratings = 0
    count = 0
    for actor in actors:
        if actor in cast_avg_rating_dict:
            actor_ratings += float(cast_top_rating_dict[actor])
            count += 1
    return (actor_ratings/count)

for mv in timer(training_movies):
    prediction.append(average_actor_rating(mv))
    
mcs.mean_squared_error(rating_vector, prediction)

100%|██████████| 83533/83533 [00:02<00:00, 34696.14it/s]


3.1685789398511224

In [42]:
for mv in test_movies:
    print ("%s: %.2f" %(mv,average_actor_rating(mv)))

Batman v Superman: Dawn of Justice (2016): 7.42
Mission: Impossible - Rogue Nation (2015): 7.49
Minions (2015): 8.38
